## NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

## Generating Sym

In [3]:
from app.templates.template_getter import get_template

c_id = 'prop'
is_raw = True

if is_raw:
    folder = f'{c_id}/t_raw'
    c_id = f'{c_id}_raw'
else:
    folder = f'{c_id}/t'

contract = get_template(c_id)
sym_spec = contract.to_sym()

FILEPATH = f'./app/templates/{folder}/{c_id}.txt'

f = open(FILEPATH, 'w')
f.write(sym_spec)
f.close()

## Test Bench

In [ ]:
from app.src.user_scripts.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [ ]:
val = 'I arrived within 2 weeks of the project being completed.'

doc = nlp(val)

summarizer.summarize(val)